# Descarga de Imagenes Satelitales de San Juan de Pasto

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/geoai/blob/main/GeoIA-Pasto/notebooks/02_Descarga_Imagenes_Satelitales_Pasto.ipynb)

---

## Objetivos

1. Descargar imagenes Sentinel-2 del area de Pasto
2. Descargar imagenes Landsat 8/9 del area de Pasto
3. Buscar datos en Microsoft Planetary Computer
4. Visualizar las imagenes descargadas

## 1. Instalacion de Dependencias

In [ ]:
# Instalacion de GeoAI y dependencias
%pip install -q geoai-py leafmap pystac-client planetary-computer rioxarray

## 2. Importacion de Librerias

In [ ]:
import geoai
import leafmap
import rioxarray as rxr
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import warnings

warnings.filterwarnings("ignore")

print(f"GeoAI version: {geoai.__version__}")

## 3. Definicion del Area de Estudio

In [ ]:
# Coordenadas de San Juan de Pasto
PASTO_CENTER = {"lat": 1.2136, "lon": -77.2811}

# Bounding Box del area urbana de Pasto
# Formato: (min_lon, min_lat, max_lon, max_lat)
PASTO_BBOX = (-77.32, 1.18, -77.24, 1.26)

# Bounding Box extendido para incluir Volcan Galeras y alrededores
PASTO_BBOX_EXTENDED = (-77.45, 1.10, -77.15, 1.35)

print(f"Area de estudio principal: {PASTO_BBOX}")
print(f"Area extendida (incluye Galeras): {PASTO_BBOX_EXTENDED}")

## 4. Busqueda de Imagenes Sentinel-2 en Planetary Computer

In [ ]:
import pystac_client
import planetary_computer

# Conectar al catalogo de Planetary Computer
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

# Parametros de busqueda
time_range = "2024-01-01/2024-12-31"
cloud_cover = 20  # Maximo porcentaje de nubes

# Buscar imagenes Sentinel-2
search = catalog.search(
    collections=["sentinel-2-l2a"],
    bbox=PASTO_BBOX,
    datetime=time_range,
    query={"eo:cloud_cover": {"lt": cloud_cover}},
)

items = list(search.items())
print(
    f"Se encontraron {len(items)} imagenes Sentinel-2 con menos de {cloud_cover}% de nubes"
)

In [ ]:
# Mostrar informacion de las primeras imagenes
if items:
    print("\nPrimeras 5 imagenes encontradas:")
    print("-" * 60)
    for i, item in enumerate(items[:5]):
        print(f"{i+1}. ID: {item.id}")
        print(f"   Fecha: {item.datetime}")
        print(f"   Nubes: {item.properties.get('eo:cloud_cover', 'N/A')}%")
        print()

## 5. Visualizar una Imagen Sentinel-2

In [ ]:
if items:
    # Seleccionar la imagen con menos nubes
    best_item = min(items, key=lambda x: x.properties.get("eo:cloud_cover", 100))

    print(f"Imagen seleccionada: {best_item.id}")
    print(f"Fecha: {best_item.datetime}")
    print(f"Cobertura de nubes: {best_item.properties.get('eo:cloud_cover')}%")

    # Mostrar bandas disponibles
    print("\nBandas disponibles:")
    for band_name, band_info in best_item.assets.items():
        if (
            "image" in band_info.media_type
            if hasattr(band_info, "media_type")
            else True
        ):
            print(f"  - {band_name}")

In [ ]:
# Crear mapa con la imagen Sentinel-2
if items:
    m = leafmap.Map(
        center=[PASTO_CENTER["lat"], PASTO_CENTER["lon"]], zoom=12, height="600px"
    )

    # Agregar imagen de vista previa
    if "visual" in best_item.assets:
        visual_url = best_item.assets["visual"].href
        m.add_cog_layer(
            visual_url, name=f"Sentinel-2 ({best_item.datetime.strftime('%Y-%m-%d')})"
        )

    m.add_basemap("OpenStreetMap")
    m.add_layer_control()
    m

## 6. Busqueda de Imagenes Landsat

In [ ]:
# Buscar imagenes Landsat
search_landsat = catalog.search(
    collections=["landsat-c2-l2"],
    bbox=PASTO_BBOX,
    datetime=time_range,
    query={"eo:cloud_cover": {"lt": cloud_cover}},
)

items_landsat = list(search_landsat.items())
print(
    f"Se encontraron {len(items_landsat)} imagenes Landsat con menos de {cloud_cover}% de nubes"
)

In [ ]:
# Mostrar informacion de imagenes Landsat
if items_landsat:
    print("\nPrimeras 5 imagenes Landsat encontradas:")
    print("-" * 60)
    for i, item in enumerate(items_landsat[:5]):
        print(f"{i+1}. ID: {item.id}")
        print(f"   Fecha: {item.datetime}")
        print(f"   Nubes: {item.properties.get('eo:cloud_cover', 'N/A')}%")
        print(f"   Plataforma: {item.properties.get('platform', 'N/A')}")
        print()

## 7. Descarga de Imagenes con GeoAI

In [ ]:
# Usar geoai para descargar imagenes de forma simplificada
# Crear directorio para guardar las imagenes
import os

output_dir = "./data/pasto_images"
os.makedirs(output_dir, exist_ok=True)

print(f"Directorio de salida: {output_dir}")

In [ ]:
# Descargar una imagen Sentinel-2 usando geoai
if items:
    best_item = min(items, key=lambda x: x.properties.get("eo:cloud_cover", 100))

    # Descargar bandas RGB (B04, B03, B02)
    bands_to_download = ["B04", "B03", "B02", "B08"]  # Red, Green, Blue, NIR

    print(f"Descargando bandas de la imagen: {best_item.id}")

    downloaded_files = []
    for band in bands_to_download:
        if band in best_item.assets:
            band_url = best_item.assets[band].href
            output_file = os.path.join(output_dir, f"pasto_{band}.tif")

            # Descargar usando rioxarray
            try:
                da = rxr.open_rasterio(band_url)
                # Recortar al bbox de Pasto
                da_clip = da.rio.clip_box(*PASTO_BBOX)
                da_clip.rio.to_raster(output_file)
                downloaded_files.append(output_file)
                print(f"  Descargado: {band} -> {output_file}")
            except Exception as e:
                print(f"  Error descargando {band}: {e}")

## 8. Visualizar Imagenes Descargadas

In [ ]:
# Verificar archivos descargados
if os.path.exists(output_dir):
    files = os.listdir(output_dir)
    print("Archivos en el directorio de salida:")
    for f in files:
        filepath = os.path.join(output_dir, f)
        size_mb = os.path.getsize(filepath) / (1024 * 1024)
        print(f"  - {f} ({size_mb:.2f} MB)")

In [ ]:
# Visualizar imagen RGB si las bandas estan disponibles
red_file = os.path.join(output_dir, "pasto_B04.tif")
green_file = os.path.join(output_dir, "pasto_B03.tif")
blue_file = os.path.join(output_dir, "pasto_B02.tif")

if all(os.path.exists(f) for f in [red_file, green_file, blue_file]):
    # Cargar bandas
    red = rxr.open_rasterio(red_file).squeeze()
    green = rxr.open_rasterio(green_file).squeeze()
    blue = rxr.open_rasterio(blue_file).squeeze()

    # Crear composicion RGB
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))

    # Normalizar valores para visualizacion
    def normalize(arr):
        arr = arr.values.astype(float)
        arr = np.clip(arr, 0, 3000)  # Recortar valores extremos
        return (arr - arr.min()) / (arr.max() - arr.min())

    rgb = np.dstack([normalize(red), normalize(green), normalize(blue)])

    axes[0].imshow(rgb)
    axes[0].set_title("Imagen RGB - San Juan de Pasto")
    axes[0].axis("off")

    # Banda NIR si esta disponible
    nir_file = os.path.join(output_dir, "pasto_B08.tif")
    if os.path.exists(nir_file):
        nir = rxr.open_rasterio(nir_file).squeeze()
        # Composicion falso color (NIR, Red, Green)
        false_color = np.dstack([normalize(nir), normalize(red), normalize(green)])
        axes[1].imshow(false_color)
        axes[1].set_title("Falso Color (NIR-R-G) - Vegetacion en rojo")
    else:
        axes[1].imshow(rgb)
        axes[1].set_title("Imagen RGB (copia)")

    axes[1].axis("off")

    plt.tight_layout()
    plt.savefig(
        os.path.join(output_dir, "pasto_composicion.png"), dpi=150, bbox_inches="tight"
    )
    plt.show()
    print("\nComposicion guardada en: pasto_composicion.png")
else:
    print("No se encontraron todas las bandas necesarias para la composicion RGB")

## 9. Mapa Interactivo con las Imagenes

In [ ]:
# Crear mapa interactivo con las imagenes descargadas
m = leafmap.Map(
    center=[PASTO_CENTER["lat"], PASTO_CENTER["lon"]], zoom=12, height="600px"
)

# Agregar capas base
m.add_basemap("OpenStreetMap")
m.add_basemap("Esri.WorldImagery")

# Agregar imagenes descargadas si existen
if os.path.exists(red_file):
    try:
        m.add_raster(red_file, layer_name="Banda Roja (B04)", colormap="Reds")
    except Exception as e:
        print(f"No se pudo agregar la banda roja: {e}")

m.add_layer_control()
m

## 10. Buscar Imagenes por Fechas Especificas

In [ ]:
def buscar_imagenes_pasto(
    fecha_inicio, fecha_fin, max_nubes=20, coleccion="sentinel-2-l2a"
):
    """
    Busca imagenes satelitales para el area de Pasto.

    Args:
        fecha_inicio: Fecha de inicio (YYYY-MM-DD)
        fecha_fin: Fecha de fin (YYYY-MM-DD)
        max_nubes: Porcentaje maximo de nubes
        coleccion: Coleccion a buscar (sentinel-2-l2a o landsat-c2-l2)

    Returns:
        list: Lista de items encontrados
    """
    catalog = pystac_client.Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1",
        modifier=planetary_computer.sign_inplace,
    )

    search = catalog.search(
        collections=[coleccion],
        bbox=PASTO_BBOX,
        datetime=f"{fecha_inicio}/{fecha_fin}",
        query={"eo:cloud_cover": {"lt": max_nubes}},
    )

    return list(search.items())


# Ejemplo: Buscar imagenes de epoca seca (Julio-Agosto 2024)
imagenes_secas = buscar_imagenes_pasto("2024-07-01", "2024-08-31", max_nubes=15)
print(f"Imagenes en epoca seca (Jul-Ago 2024): {len(imagenes_secas)}")

# Ejemplo: Buscar imagenes de epoca lluviosa (Octubre-Noviembre 2024)
imagenes_lluviosas = buscar_imagenes_pasto("2024-10-01", "2024-11-30", max_nubes=30)
print(f"Imagenes en epoca lluviosa (Oct-Nov 2024): {len(imagenes_lluviosas)}")

## 11. Resumen

En este notebook hemos aprendido a:

1. **Conectar con Planetary Computer** para buscar imagenes satelitales
2. **Buscar imagenes Sentinel-2** del area de Pasto con filtros de nubes
3. **Buscar imagenes Landsat** como alternativa
4. **Descargar bandas especificas** (RGB y NIR)
5. **Visualizar composiciones** de color verdadero y falso color
6. **Crear mapas interactivos** con las imagenes

### Proximos pasos:

- En el siguiente notebook calcularemos indices de vegetacion (NDVI) usando estas imagenes
- Aprenderemos a detectar cambios en la cobertura vegetal de Pasto